In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
df.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-01-22T16:55:53.000Z,NaN,1,8.0,1998-02-07T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,4.0,0,10.0,1.0,1963-01-01T00:00:00.000Z,1997-01-11T00:00:00.000Z,NaN,NaN,NaN,0,7,0,200000.0,50000.0,1998,0,382.0,0,0.00,937.0,1,N,NaN,0,0.0,NaN,0.0,None,612.0,NaN,0.0,0,Pineapple Express - Southern,15000.0,0.0,60294.0,None,"OCEANSIDE, CITY OF",97.0,1.0,1.0,937.0,NaN,A,0,0.0,None,NaN,0,CA,Currently Unavailable,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,4053d562-94c7-4155-9d60-3030f34a7abc
1,0,2020-01-22T16:55:53.000Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,NaN,1.0,1967-07-01T00:00:00.000Z,1990-07-12T00:00:00.000Z,NaN,NaN,NaN,0,7,0,100000.0,40000.0,2005,1,NaN,0,0.00,NaN,1,N,NaN,0,0.0,NaN,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,None,NEW ORLEANS/ORLEANS PARISH*,6.0,6.0,1.0,NaN,NaN,A,0,0.0,None,NaN,0,LA,Currently Unavailable,70131.0,22071.0,2.207100e+10,2.207100e+11,29.9,-90.0,4094445e-bc54-42b2-a60d-b949bdc2df3a
2,0,2020-01-22T16:55:53.000Z,NaN,1,9.0,1998-09-28T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,10.0,1.0,1972-01-01T00:00:00.000Z,1997-07-24T00:00:00.000Z,8813.21,1720.0,0.0,0,1,0,100000.0,50000.0,1998,1,9313.0,0,8813.21,80000.0,1,N,2220.0,0,1720.0,0.0,0.0,None,133.0,NaN,0.0,0,Hurricane Georges (Panhandle),15000.0,0.0,120274.0,None,SANTA ROSA COUNTY *,NaN,NaN,1.0,100000.0,0.0,A,0,0.0,None,NaN,0,FL,Currently Unavailable,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,659ba8e0-1bb3-4442-ad57-3f384db43176
3,0,2019-09-19T13:45:58.000Z,1.0,1,9.0,1994-10-07T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,10.0,1.0,1960-01-01T00:00:00.000Z,1993-10-01T00:00:00.000Z,2906.00,0.0,0.0,0,7,0,100000.0,25000.0,1994,0,4428.0,0,2906.00,100000.0,1,N,NaN,0,0.0,NaN,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,450026.0,None,"BEAUFORT, CITY OF",97.0,NaN,1.0,0.0,NaN,A,0,0.0,None,NaN,0,SC,Currently Unavailable,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,97063578-59b6-403d-b9c3-2e70fd8b6f38
4,0,2019-09-19T13:45:58.000Z,NaN,1,8.0,1996-03-11T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1988-01-01T00:00:00.000Z,1996-01-11T00:00:00.000Z,3875.53,1545.0,0.0,1,7,0,100000.0,25000.0,1996,1,5252.0,0,3875.53,100000.0,1,N,3115.0,0,1545.0,0.0,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,125092.0,None,BREVARD COUNTY *,NaN,NaN,1.0,0.0,0.0,A,0,0.0,None,1.0,0,FL,Currently Unavailable,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,613c20ef-7838-48ee-a257-07a49e997d3a


In [4]:
#fill na values with 0

df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].fillna(0)
df_geographic_unique['countyCode'] = df_geographic_unique['countyCode'].fillna(0)
df_geographic_unique['censusTract'] = df_geographic_unique['censusTract'].fillna(0)
df_geographic_unique['censusBlockGroupFips'] = df_geographic_unique['censusBlockGroupFips'].fillna(0)
df_geographic_unique['latitude'] = df_geographic_unique['latitude'].fillna(0)
df_geographic_unique['longitude'] = df_geographic_unique['longitude'].fillna(0)

df_geographic_unique = df_geographic_unique.drop_duplicates()

In [5]:
#convert the units to string to use mapping and fix the lenght of each units

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].astype(int).astype(str)
df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

df_geographic_unique['countyCode'] = [str(int(float(i))) for i in df_geographic_unique['countyCode']]
df_geographic_unique['countyCode'] = [censusBG.zfill(5) for censusBG in df_geographic_unique['countyCode']]

df_geographic_unique['censusTract'] = [str(int(float(i))) for i in df_geographic_unique['censusTract']]
df_geographic_unique['censusTract'] = [censusBG.zfill(11) for censusBG in df_geographic_unique['censusTract']]

In [6]:
# Define bins and labels for yearOfLoss_1990_2021

bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)


df_geographic_unique = df_geographic_unique.drop_duplicates()

In [7]:
import pandas as pd

# Step 1: Define the bins.
# Adjusting the bins so that 2011 falls into the 2012 bin
custom_bins = [0, 2000, 2010, 2012] + list(range(2013, 2023)) + [2024]

# Step 2: Define the labels.
# Adjusting the labels to reflect the new bin structure
custom_labels = [0, 2000, 2010] + list(range(2012, 2023))  # The label 2022 will apply to both 2022 and 2023.

# Ensure the number of labels is one less than the number of bin edges.
assert len(custom_bins) == len(custom_labels) + 1, "Number of bin labels must be one fewer than the number of bin edges."

# Step 3: Apply the custom binning and create the 'zip_year_bin' column.
df_geographic_unique['zip_year_bin'] = pd.cut(
    df_geographic_unique['yearOfLoss'],
    bins=custom_bins,
    labels=custom_labels,
    right=False,  # Ensuring the rightmost edge is exclusive.
    include_lowest=True,  # Including the leftmost edge.
).astype(int)  # Converting the resulting categories to integers for convenience.

## Read shapefiles

In [8]:
#State shapefile

states = pygris.states()

state_df = states[['STUSPS', 'NAME', 'geometry']]

Using the default year of 2021


In [9]:
#Lat-Long shapefile
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [10]:
# Blockgroup shapefile

chunk_size = 40000 
chunks = [x for x in range(0, 120000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df= pd.concat(gdf_list, ignore_index=True)

In [11]:
chunk_size = 50000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 300000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

In [12]:
# Read shapefile of county code
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [13]:
# Read shapefile of census tract code
chunk_size = 30000 
chunks = [x for x in range(0, 60000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/Tract_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
Tract_df= pd.concat(gdf_list, ignore_index=True)

In [ ]:
###Read the union shapefiles for zipcodes

### Mapping Geometries to the Dataset

In [14]:
state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)
County_df.rename(columns={'geometry': 'geometry_county'}, inplace=True)
Tract_df.rename(columns={'geometry': 'geometry_tract'}, inplace=True)

In [15]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

lat_long_df.reset_index(inplace=True)

In [16]:
# Initial mapping with multi-index
BG_df.set_index(['GEOID'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips']).index.map(BG_df['geometry_BG'])

BG_df.reset_index(inplace=True)

In [17]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'zip_year_bin']).index.map(zipcode_df['geometry_zipcode'])

zipcode_df.reset_index(inplace=True)

In [18]:
# Initial mapping with multi-index
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

state_df.reset_index(inplace=True)

In [19]:
# Initial mapping with multi-index
County_df.set_index(['CountyID'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode']).index.map(County_df['geometry_county'])

# Resetting the index of County_df (return to multi-index)
County_df.reset_index(inplace=True)

In [20]:
# Initial mapping with multi-index
Tract_df.set_index(['censusTractID'], inplace=True)
df_geographic_unique['geometry_tract'] = df_geographic_unique.set_index(['censusTract']).index.map(Tract_df['geometry_tract'])

# Resetting the index of County_df (return to multi-index)
Tract_df.reset_index(inplace=True)

### Adding the inconsistent flags to the dataset

In [21]:
df_1_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_1.parquet.gzip')
df_2_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_2.parquet.gzip')
df_3_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_3.parquet.gzip')
df_4_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_4.parquet.gzip')
df_5_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_5.parquet.gzip')
df_6_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_6.parquet.gzip')

In [22]:
(sum((df_1_inconsistent['zipInconsistent'] == 1.0) & (df_1_inconsistent['oneWrong'] == 1.0)) + 
 sum((df_3_inconsistent['zipInconsistent'] == 1.0) & (df_3_inconsistent['oneWrong'] == 1.0)) +
 sum((df_5_inconsistent['zipInconsistent'] == 1.0) & (df_5_inconsistent['oneWrong'] == 1.0)))

5702

In [23]:
(sum((df_1_inconsistent['stateInconsistent'] == 1.0) & (df_1_inconsistent['oneWrong'] == 1.0)) + 
  sum((df_2_inconsistent['stateInconsistent'] == 1.0) & (df_2_inconsistent['oneWrong'] == 1.0)) +
 sum((df_3_inconsistent['stateInconsistent'] == 1.0) & (df_3_inconsistent['oneWrong'] == 1.0)) +
  sum((df_4_inconsistent['stateInconsistent'] == 1.0) & (df_4_inconsistent['oneWrong'] == 1.0)) +
 sum((df_5_inconsistent['stateInconsistent'] == 1.0) & (df_5_inconsistent['oneWrong'] == 1.0)))

2774

In [24]:
#df_2_inconsistent
df_2_inconsistent['reportedZipCode'] = df_2_inconsistent['reportedZipCode'].fillna(0)

#df_3_inconsistent
df_3_inconsistent['countyCode'] = df_3_inconsistent['countyCode'].fillna(0)

#df_4_inconsistent
df_4_inconsistent['reportedZipCode'] = df_4_inconsistent['reportedZipCode'].fillna(0)
df_4_inconsistent['countyCode'] = df_4_inconsistent['countyCode'].fillna(0)

#df_5_inconsistent
df_5_inconsistent['censusTract'] = df_5_inconsistent['censusTract'].fillna(0)
df_5_inconsistent['censusBlockGroupFips'] = df_5_inconsistent['censusBlockGroupFips'].fillna(0)

In [25]:
#convert the units to string to use mapping and fix the lenght of each units
#df_2_inconsistent
df_2_inconsistent['reportedZipCode'] = df_2_inconsistent['reportedZipCode'].astype(int).astype(str)
df_2_inconsistent['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_2_inconsistent['reportedZipCode']]

#df_3_inconsistent
df_3_inconsistent['countyCode'] = [str(int(float(i))) for i in df_3_inconsistent['countyCode']]
df_3_inconsistent['countyCode'] = [censusBG.zfill(5) for censusBG in df_3_inconsistent['countyCode']]

#df_4_inconsistent
df_4_inconsistent['reportedZipCode'] = df_4_inconsistent['reportedZipCode'].astype(int).astype(str)
df_4_inconsistent['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_4_inconsistent['reportedZipCode']]

df_4_inconsistent['countyCode'] = [str(int(float(i))) for i in df_4_inconsistent['countyCode']]
df_4_inconsistent['countyCode'] = [censusBG.zfill(5) for censusBG in df_4_inconsistent['countyCode']]

#df_5_inconsistent
df_5_inconsistent['censusBlockGroupFips'] = [str(int(float(i))) for i in df_5_inconsistent['censusBlockGroupFips']]
df_5_inconsistent['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_5_inconsistent['censusBlockGroupFips']]

df_5_inconsistent['censusTract'] = [str(int(float(i))) for i in df_5_inconsistent['censusTract']]
df_5_inconsistent['censusTract'] = [censusBG.zfill(11) for censusBG in df_5_inconsistent['censusTract']]

In [26]:
df_1_inconsistent.drop(columns=['zip_geometry','lat_long_geometry','bg_geometry'], inplace=True)
df_2_inconsistent.drop(columns=['lat_long_geometry','state_geometry','county_geometry'], inplace=True)
df_3_inconsistent.drop(columns=['state_geometry','zip_geometry'], inplace=True)

In [30]:
df_1_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year', 'year_zipcode'], inplace=True)

In [31]:
# Initial mapping with multi-index
df_1_inconsistent.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year', 'year_zipcode'], inplace=True)

df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 
                                'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin'], inplace=True)

df_geographic_unique['zipInconsistent'] = df_geographic_unique.index.map(df_1_inconsistent['zipInconsistent'])

# Reset the index
df_geographic_unique.reset_index(inplace=True)


df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['zipInconsistent'])
df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['cbgInconsistent'])
df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['tractInconsistent'])
df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['stateInconsistent'])
df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['countyInconsistent'])
df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['latlongInconsistent'])
df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['multiple'])
df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index.map(df_1_inconsistent['oneWrong'])

# Resetting the index (return to multi-index)
df_1_inconsistent.reset_index(inplace=True)

In [32]:
df_2_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year'], inplace=True)

In [33]:
# Initial mapping with multi-index
df_2_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index
cbgInconsistent = pd.Series(multi_index.map(df_2_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_2_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_2_inconsistent['stateInconsistent']), index=multi_index)
countyInconsistent = pd.Series(multi_index.map(df_2_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_2_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_2_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_2_inconsistent['oneWrong']), index=multi_index)

# Revert df_2_inconsistent back to its original form
df_2_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021'], inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [34]:
# # Initial mapping with multi-index
# df_2_inconsistent.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_2_inconsistent.reset_index(inplace=True)

In [35]:
df_3_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year', 'year_zipcode'], inplace=True)

In [36]:
# Initial mapping with multi-index
df_3_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year', 'year_zipcode'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index
zipInconsistent = pd.Series(multi_index.map(df_3_inconsistent['zipInconsistent']), index=multi_index)
cbgInconsistent = pd.Series(multi_index.map(df_3_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_3_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_3_inconsistent['stateInconsistent']), index=multi_index)
# countyInconsistent = pd.Series(multi_index.map(df_3_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_3_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_3_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_3_inconsistent['oneWrong']), index=multi_index)

# Revert df_3_inconsistent back to its original form
df_3_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin'], inplace=True)
df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
# df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [37]:
# # Initial mapping with multi-index
# df_3_inconsistent.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'year'], inplace=True)

# df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['zipInconsistent'])
# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['stateInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_3_inconsistent.reset_index(inplace=True)

In [38]:
df_4_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year'], inplace=True)

In [39]:
# Initial mapping with multi-index
df_4_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index
# zipInconsistent = pd.Series(multi_index.map(df_4_inconsistent['zipInconsistent']), index=multi_index)
cbgInconsistent = pd.Series(multi_index.map(df_4_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_4_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_4_inconsistent['stateInconsistent']), index=multi_index)
# countyInconsistent = pd.Series(multi_index.map(df_3_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_4_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_4_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_4_inconsistent['oneWrong']), index=multi_index)

# Revert df_4_inconsistent back to its original form
df_4_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021'], inplace=True)
# df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
# df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [40]:
# # Initial mapping with multi-index
# df_4_inconsistent.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['stateInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_4_inconsistent.reset_index(inplace=True)

In [41]:
df_5_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year','year_zipcode'], inplace=True)

In [42]:
# Initial mapping with multi-index
df_5_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year', 'year_zipcode'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin']).index
zipInconsistent = pd.Series(multi_index.map(df_5_inconsistent['zipInconsistent']), index=multi_index)
# cbgInconsistent = pd.Series(multi_index.map(df_5_inconsistent['cbgInconsistent']), index=multi_index)
# tractInconsistent = pd.Series(multi_index.map(df_5_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_5_inconsistent['stateInconsistent']), index=multi_index)
countyInconsistent = pd.Series(multi_index.map(df_5_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_5_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_5_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_5_inconsistent['oneWrong']), index=multi_index)

# Revert df_5_inconsistent back to its original form
df_5_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021', 'zip_year_bin'], inplace=True)
df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
# df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
# df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [43]:
# # Initial mapping with multi-index
# df_5_inconsistent.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'year'], inplace=True)

# df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['zipInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_5_inconsistent.reset_index(inplace=True)

In [44]:
# # Initial mapping with multi-index (df_6_inconsistent is empty)
# df_6_inconsistent.set_index(['state', 'countyCode',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['latlongInconsistent'])

# # Resetting the index of County_df (return to multi-index)
# df_6_inconsistent.reset_index(inplace=True)

In [45]:
df_geographic_unique[df_geographic_unique['zipInconsistent'].notna()].head()

,reportedZipCode,censusTract,state,countyCode,censusBlockGroupFips,latitude,longitude,yearOfLoss_1990_2021,zip_year_bin,yearOfLoss,geometry_lat_long,geometry_BG,geometry_zipcode,geometry_state,geometry_county,geometry_tract,zipInconsistent,cbgInconsistent,tractInconsistent,stateInconsistent,countyInconsistent,latlongInconsistent,multiple_inconsistent,single_inconsistent
31,28646,37011930200,NC,37011,370119302003,36.0,-81.9,2000,2000,2004,"POLYGON ((-81.95000 35.95000, -81.85000 35.950...","POLYGON ((-81.88422 36.05066, -81.88433 36.050...","MULTIPOLYGON (((-81.85616 36.08761, -81.85614 ...","MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ...","POLYGON ((-82.01657 36.14514, -82.01645 36.145...","POLYGON ((-81.92090 36.06335, -81.92059 36.063...",1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
154,77550,48167726100,TX,48167,481677261001,29.2,-95.0,2000,2000,2008,"POLYGON ((-95.05000 29.15000, -94.95000 29.150...","POLYGON ((-95.02134 29.23417, -95.02120 29.234...","POLYGON ((-94.81155 29.27683, -94.81171 29.277...","MULTIPOLYGON (((-94.71830 29.72885, -94.71721 ...","POLYGON ((-95.11995 29.29611, -95.12532 29.304...","POLYGON ((-95.12993 29.08562, -95.12517 29.093...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
159,77553,48167723900,TX,48167,481677239002,29.4,-94.7,2000,2000,2008,"POLYGON ((-94.75000 29.35000, -94.65000 29.350...","POLYGON ((-94.78514 29.35712, -94.78309 29.359...","POLYGON ((-94.80275 29.34850, -94.80102 29.347...","MULTIPOLYGON (((-94.71830 29.72885, -94.71721 ...","POLYGON ((-95.11995 29.29611, -95.12532 29.304...","POLYGON ((-94.56990 29.57321, -94.56982 29.573...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
378,32503,12033002500,FL,12033,120330025001,30.3,-87.1,2000,2000,2004,"POLYGON ((-87.15000 30.25000, -87.05000 30.250...","POLYGON ((-87.19211 30.32403, -87.19176 30.324...","POLYGON ((-87.19960 30.45104, -87.19951 30.451...","MULTIPOLYGON (((-80.62717 25.00040, -80.62600 ...","POLYGON ((-87.44034 30.69035, -87.44038 30.690...","POLYGON ((-86.91913 30.36765, -86.91909 30.366...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
379,32503,12033002500,FL,12033,120330025001,30.3,-87.1,2000,2000,2005,"POLYGON ((-87.15000 30.25000, -87.05000 30.250...","POLYGON ((-87.19211 30.32403, -87.19176 30.324...","POLYGON ((-87.19960 30.45104, -87.19951 30.451...","MULTIPOLYGON (((-80.62717 25.00040, -80.62600 ...","POLYGON ((-87.44034 30.69035, -87.44038 30.690...","POLYGON ((-86.91913 30.36765, -86.91909 30.366...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [46]:
test = df_geographic_unique[df_geographic_unique['single_inconsistent'] == 1.0]

print(f"zipInconsistent: {sum(test['zipInconsistent'] == 1.0)}")
print(f"cbgInconsistent: {sum(test['cbgInconsistent'] == 1.0)}")
print(f"tractInconsistent: {sum((test['tractInconsistent'] == 1.0))}")
print(f"stateInconsistent: {sum((test['stateInconsistent'] == 1.0))}")
print(f"countyInconsistent: {sum((test['countyInconsistent'] == 1.0))}")
print(f"latlongInconsistent: {sum((test['latlongInconsistent'] == 1.0))}")
print(f"multiple_inconsistent: {sum((test['multiple_inconsistent'] == 1.0))}")
print(f"single_inconsistent: {sum((test['single_inconsistent'] == 1.0))}")

zipInconsistent: 5702
cbgInconsistent: 24
tractInconsistent: 0
stateInconsistent: 3658
countyInconsistent: 3
latlongInconsistent: 45
multiple_inconsistent: 0
single_inconsistent: 9432


In [47]:
print(f"zipInconsistent: {sum((df_geographic_unique['zipInconsistent'].notna()) & (df_geographic_unique['zipInconsistent'] == 1.0))}")
print(f"cbgInconsistent: {sum((df_geographic_unique['cbgInconsistent'].notna()) & (df_geographic_unique['cbgInconsistent'] == 1.0))}")
print(f"tractInconsistent: {sum((df_geographic_unique['tractInconsistent'].notna()) & (df_geographic_unique['tractInconsistent'] == 1.0))}")
print(f"stateInconsistent: {sum((df_geographic_unique['stateInconsistent'].notna()) & (df_geographic_unique['stateInconsistent'] == 1.0))}")
print(f"countyInconsistent: {sum((df_geographic_unique['countyInconsistent'].notna()) & (df_geographic_unique['countyInconsistent'] == 1.0))}")
print(f"latlongInconsistent: {sum((df_geographic_unique['latlongInconsistent'].notna()) & (df_geographic_unique['latlongInconsistent'] == 1.0))}")
print(f"multiple_inconsistent: {sum((df_geographic_unique['multiple_inconsistent'].notna()) & (df_geographic_unique['multiple_inconsistent'] == 1.0))}")
print(f"single_inconsistent: {sum((df_geographic_unique['single_inconsistent'].notna()) & (df_geographic_unique['single_inconsistent'] == 1.0))}")

zipInconsistent: 7019
cbgInconsistent: 755
tractInconsistent: 5
stateInconsistent: 3686
countyInconsistent: 7
latlongInconsistent: 815
multiple_inconsistent: 1253
single_inconsistent: 9432


In [48]:
#TODO: for all NA values change it to 1
#TODO: create 'multipleInconsistent' column where add all flags
#TODO: for state = 'UN', make it N
#TODO (Not here but later): for missing geometries, change flag to M

In [49]:
df_geographic_unique['zipInconsistent'].value_counts()

1.0    7019
0.0     145
Name: zipInconsistent, dtype: int64

In [50]:
df_geographic_unique.head()

,reportedZipCode,censusTract,state,countyCode,censusBlockGroupFips,latitude,longitude,yearOfLoss_1990_2021,zip_year_bin,yearOfLoss,geometry_lat_long,geometry_BG,geometry_zipcode,geometry_state,geometry_county,geometry_tract,zipInconsistent,cbgInconsistent,tractInconsistent,stateInconsistent,countyInconsistent,latlongInconsistent,multiple_inconsistent,single_inconsistent
0,92056,06073018512,CA,06073,060730185123,33.2,-117.3,1990,0,1998,"POLYGON ((-117.35000 33.15000, -117.25000 33.1...","POLYGON ((-117.33340 33.20936, -117.33334 33.2...",None,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...","POLYGON ((-117.43743 33.17953, -117.44954 33.1...","POLYGON ((-117.32205 33.21222, -117.32215 33.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70131,22071000616,LA,22071,220710006163,29.9,-90.0,2000,2000,2005,"POLYGON ((-90.05000 29.85000, -89.95000 29.850...","POLYGON ((-90.00122 29.92086, -90.00119 29.921...","POLYGON ((-90.01051 29.94005, -90.00980 29.939...","MULTIPOLYGON (((-88.86770 29.86155, -88.86566 ...","POLYGON ((-89.95236 29.90007, -89.95321 29.900...","POLYGON ((-90.00180 29.91346, -90.00172 29.913...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32566,12113010815,FL,12113,121130108152,30.4,-86.9,1990,0,1998,"POLYGON ((-86.95000 30.35000, -86.85000 30.350...","POLYGON ((-87.03397 30.38297, -87.03395 30.382...",None,"MULTIPOLYGON (((-80.62717 25.00040, -80.62600 ...","POLYGON ((-87.29836 30.82735, -87.29860 30.827...","POLYGON ((-87.03494 30.38472, -87.03490 30.384...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29902,45013000700,SC,45013,450130007002,32.4,-80.7,1990,0,1994,"POLYGON ((-80.75000 32.35000, -80.65000 32.350...","POLYGON ((-80.68993 32.43298, -80.68975 32.432...",None,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...","POLYGON ((-80.81918 32.48594, -80.81943 32.487...","POLYGON ((-80.70786 32.41190, -80.70783 32.412...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32940,12009063107,FL,12009,120090631073,28.3,-80.7,1990,0,1996,"POLYGON ((-80.75000 28.25000, -80.65000 28.250...","POLYGON ((-80.71440 28.24480, -80.71391 28.244...",None,"MULTIPOLYGON (((-80.62717 25.00040, -80.62600 ...","POLYGON ((-80.49977 28.40954, -80.49989 28.409...","POLYGON ((-80.71826 28.26372, -80.71821 28.264...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
count_none =(df_geographic_unique['geometry_lat_long'] == None).sum()
print(count_none)

4181


In [52]:
def is_empty_or_none(geometry):
    return geometry is None or geometry.is_empty

In [53]:
assignment_counts = {
    "bg_zip_not_empty": 0,
    "bg_no_zip_before_2000": 0,
    "no_bg_zip_after_2000": 0,
    "no_bg_no_zip": 0,
    "lat_long_none": 0,
    "lat_long_bg_no_zip_before_2000_1": 0,
    "lat_long_bg_no_zip_before_2000_2": 0,
    "lat_long_bg_zip_after_2000": 0,
    "lat_long_no_bg_zip_after_2000":0,
    "final_lat_long": 0,
    "lat_long_zip_after_2000" : 0,
    "lat_long_only_1" :0
}

### Creating new geometry

In [54]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract','censusBlockGroupFips', 'latitude', 
                                        'longitude', 'state', 'new_geographic_unit', 'year', 'zip_year_bin', 'geometry_zipcode',
                                       'geometry_county', 'geometry_tract','geometry_BG','geometry_lat_long','geometry_state', 
                                        'geometry_new_unit', 'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 
                                        'stateInconsistent', 'latlongInconsistent', 'zipcodeInconsistent','multipleinconsistent','singleinconsistent'])

empty = Polygon()

# Iterate through each row in BG_df_1990 and each row in lat_long_df to find intersections
for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_1990_2021']
    year_zipcode = row_unit['zip_year_bin']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    cbgInconsistent = row_unit['cbgInconsistent']
    tractInconsistent = row_unit['tractInconsistent']
    countyInconsistent = row_unit['countyInconsistent']
    stateInconsistent = row_unit['stateInconsistent']
    latlongInconsistent = row_unit['latlongInconsistent']
    zipcodeInconsistent = row_unit['zipInconsistent']
    multipleInconsistent = row_unit['multiple_inconsistent']
    singleInconsistent = row_unit['single_inconsistent']
    new_geographic_unit = '0'
    intersection_geometry = empty

    if (singleInconsistent == 1) and (latlongInconsistent == 1):

        if (bg_geometry is not None) and (zipcode_geometry is not None) and (year >= 2000):
            intersection_geometry = bg_geometry.intersection(zipcode_geometry)
            
            if intersection_geometry.is_empty:
                print("OH NO!!")
                break
            else:
                print("bg zip not empty")
                new_geographic_unit = f"{'0'*4}{'0'*4}{bg_id:12}{zipcode:5}{year_zipcode:4}"            
                assignment_counts["bg_zip_not_empty"] += 1
                
        elif (bg_geometry is not None) and ((zipcode_geometry is None) or (year < 2000)):
            new_geographic_unit = f"{'0'*4}{'0'*4}{bg_id:12}{'0'*5}{'0'*4}"
            assignment_counts["bg_no_zip_before_2000"] += 1
            intersection_geometry = bg_geometry
                
        elif (bg_geometry is None) and (zipcode_geometry is not None) and (year >= 2000):
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{zipcode:5}{year_zipcode:4}"
            assignment_counts["no_bg_zip_after_2000"] += 1
            intersection_geometry = zipcode_geometry

        else:
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["no_bg_no_zip"] += 1
            intersection_geometry = empty
                
            
            
    else:     
  
        # Check for the lat_long_geometry being None
        if lat_long_geometry is None:
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["lat_long_none"] += 1
            intersection_geometry = empty
    
        elif (bg_geometry is None and (zipcode_geometry is None or (zipcode_geometry is not None and year < 2000))):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["lat_long_only_1"] += 1
            intersection_geometry = lat_long_geometry

        elif ((zipcode_geometry is None or (zipcode_geometry is not None and year < 2000)) and not is_empty_or_none(lat_long_geometry.intersection(bg_geometry))):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{bg_id:12}{'0'*5}{'2010':4}"
            assignment_counts["lat_long_bg_no_zip_before_2000_1"] += 1
            intersection_geometry = lat_long_geometry.intersection(bg_geometry)

        elif (bg_geometry is None and zipcode_geometry is not None and year >= 2000 and not is_empty_or_none(lat_long_geometry.intersection(zipcode_geometry))):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{zipcode:5}{year_zipcode:4}"
            assignment_counts["lat_long_zip_after_2000"] += 1
            intersection_geometry = lat_long_geometry.intersection(zipcode_geometry)

    
        elif (bg_geometry is not None and zipcode_geometry is not None and year >= 2000):
            if not is_empty_or_none(lat_long_geometry.intersection(bg_geometry).intersection(zipcode_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{bg_id:12}{zipcode:5}{year_zipcode:4}"
                assignment_counts["lat_long_bg_zip_after_2000"] += 1
                intersection_geometry = lat_long_geometry.intersection(bg_geometry).intersection(zipcode_geometry)
        
            elif not is_empty_or_none(lat_long_geometry.intersection(bg_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
                assignment_counts["lat_long_bg_no_zip_before_2000_2"] +=1
                intersection_geometry = lat_long_geometry.intersection(bg_geometry)
        
            elif not is_empty_or_none(lat_long_geometry.intersection(zipcode_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{zipcode:5}{year_zipcode:4}"
                assignment_counts['lat_long_no_bg_zip_after_2000'] +=1
                intersection_geometry = lat_long_geometry.intersection(zipcode_geometry)
        else:
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
                assignment_counts['final_lat_long'] +=1
                intersection_geometry = lat_long_geometry

    
    new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
        'reportedZipCode': [zipcode],
        'countyCode': [county_id],
        'censusTract': [tract_id],
        'censusBlockGroupFips': [bg_id],
        'latitude': [lat],
        'longitude': [long],
        'state': [state],
        'new_geographic_unit': [new_geographic_unit],
        'year': [year],
        'year_zipcode': [year_zipcode],
        'geometry_zipcode': [zipcode_geometry],
        'geometry_county': [county_geometry],
        'geometry_tract': [tract_geometry],
        'geometry_BG': [bg_geometry],
        'geometry_lat_long': [lat_long_geometry],
        'geometry_state': [state_geometry],
        'geometry_new_unit': [intersection_geometry],
        'cbgInconsistent': [cbgInconsistent],
        'tractInconsistent': [tractInconsistent],
        'countyInconsistent': [countyInconsistent],
        'stateInconsistent': [stateInconsistent],
        'latlongInconsistent': [latlongInconsistent],
        'zipcodeInconsistent': [zipcodeInconsistent],
        'multipleInconsistent': [multipleInconsistent],
        'singleInconsistent': [singleInconsistent]
        })], ignore_index=True)

bg zip not empty


KeyboardInterrupt: 

In [55]:
new_unit_df

,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,state,new_geographic_unit,year,zip_year_bin,geometry_zipcode,geometry_county,geometry_tract,geometry_BG,geometry_lat_long,geometry_state,geometry_new_unit,cbgInconsistent,tractInconsistent,countyInconsistent,stateInconsistent,latlongInconsistent,zipcodeInconsistent,multipleinconsistent,singleinconsistent,year_zipcode,multipleInconsistent,singleInconsistent
0,92056,06073,06073018512,060730185123,33.2,-117.3,CA,033211173060730185123000002010,1990,NaN,None,"POLYGON ((-117.437426 33.17953, -117.449536 33...","POLYGON ((-117.322047 33.212215, -117.322155 3...","POLYGON ((-117.333399 33.209356, -117.33333999...","POLYGON ((-117.35 33.150000000000006, -117.25 ...","MULTIPOLYGON (((-118.604415 33.478552, -118.59...","POLYGON ((-117.33333999999999 33.211272, -117....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,70131,22071,22071000616,220710006163,29.9,-90.0,LA,02991900220710006163701312000,2000,NaN,"POLYGON ((-90.01051199999999 29.940051, -90.00...",POLYGON ((-89.95236299999999 29.90006799999999...,"POLYGON ((-90.00180499999999 29.913465, -90.00...","POLYGON ((-90.001221 29.920858, -90.001195 29....","POLYGON ((-90.05 29.849999999999998, -89.95 29...","MULTIPOLYGON (((-88.8677 29.861551, -88.865659...","POLYGON ((-90.001161 29.922204999999998, -90.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,NaN,NaN
2,32566,12113,12113010815,121130108152,30.4,-86.9,FL,03041869121130108152000002010,1990,NaN,None,"POLYGON ((-87.298363 30.82735, -87.298603 30.8...","POLYGON ((-87.034937 30.384718999999997, -87.0...","POLYGON ((-87.033974 30.382972, -87.033948 30....","POLYGON ((-86.95 30.349999999999998, -86.85000...","MULTIPOLYGON (((-80.627171 25.000401, -80.6260...","POLYGON ((-86.95 30.402405847176077, -86.94986...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,29902,45013,45013000700,450130007002,32.4,-80.7,SC,03241807450130007002000002010,1990,NaN,None,"POLYGON ((-80.81917899999999 32.485942, -80.81...","POLYGON ((-80.707863 32.411901, -80.7078279999...","POLYGON ((-80.68992899999999 32.43298, -80.689...","POLYGON ((-80.75 32.35, -80.65 32.35, -80.65 3...","MULTIPOLYGON (((-79.507948 33.020076, -79.5071...","POLYGON ((-80.68974899999999 32.432969, -80.68...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,32940,12009,12009063107,120090631073,28.3,-80.7,FL,02831807120090631073000002010,1990,NaN,None,"POLYGON ((-80.499771 28.409543, -80.4998859999...","POLYGON ((-80.71825799999999 28.263724, -80.71...","POLYGON ((-80.714401 28.244802, -80.71391 28.2...","POLYGON ((-80.75 28.25, -80.65 28.25, -80.65 2...","MULTIPOLYGON (((-80.627171 25.000401, -80.6260...","POLYGON ((-80.70696706731317 28.25, -80.707043...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29460,08075,34005,34005700703,340057007032,40.0,-75.0,NJ,04001750340057007032080752012,2010,NaN,"POLYGON ((-74.9621 39.997049, -74.964841999999...","POLYGON ((-74.899153 40.073205, -74.8987949999...","POLYGON ((-74.96593399999999 40.043796, -74.96...","POLYGON ((-74.965746 40.037537, -74.965031 40....","POLYGON ((-75.05 39.95, -74.95 39.95, -74.95 4...","MULTIPOLYGON (((-74.042196 40.699966, -74.0389...","POLYGON ((-74.964822 40.038064999999996, -74.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.0,NaN,NaN
29461,33702,12103,12103024411,121030244112,27.9,-82.6,FL,02791826121030244112000002010,1990,NaN,None,"POLYGON ((-82.875 27.794342999999998, -82.8755...","POLYGON ((-82.638817 27.862136, -82.638815 27....","POLYGON ((-82.638817 27.862136, -82.638815 27....",POLYGON ((-82.64999999999999 27.84999999999999...,"MULTIPOLYGON (((-80.627171 25.000401, -80.6260...",POLYGON ((-82.6342228409894 27.849999999999998...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
29462,60076,17031,17031807500,170318075001,42.0,-87.7,IL,04201877170318075001600762000,2000,NaN,"POLYGON ((-87.723379 42.046247, -87.723382 42....","POLYGON ((-87.7927929

In [ ]:
new_unit_df1 = new_unit_df[['geometry_new_unit','new_geographic_unit']]

new_unit_df1 = new_unit_df1.drop_duplicates()

chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(new_unit_df1), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = new_unit_df1.iloc[start:end].copy()
    temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"new_geographic_unit_geometry_{start}_{end}.parquet.gzip", compression='gzip')

## Visualization

In [ ]:
print(assignment_counts)

In [ ]:
len(df_geographic_unique)

In [ ]:
total = sum(assignment_counts.values())
print(total)

In [ ]:
new_unit_df.columns

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the current CRS to WGS 84
gdf.crs = "EPSG:4326"

# Filter for only the continental US using bounding box constraints
continental_US = gdf.cx[-125:-66, 24:49]

# Project the filtered GeoDataFrame to Web Mercator (EPSG:3857)
continental_US_projected = continental_US.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = continental_US_projected.geometry.area.mean()
median_area_m2 = continental_US_projected.geometry.area.median()

# Convert the areas from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
median_area_km2 = median_area_m2 / 1e6

print(f"Average Area: {average_area_km2} square kilometers")
print(f"Median Area: {median_area_km2} square kilometers")

# Plot the geometries using the 'Reds' colormap
continental_US_projected.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Continental US: Using Latitude-Longitude geometries')

plt.show()



In [ ]:

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Project the GeoDataFrame to a metric CRS, e.g., Web Mercator (EPSG:3857)
gdf_projected = gdf.to_crs(epsg=3857)

# Calculate the areas in square meters
gdf_projected['area_m2'] = gdf_projected['geometry_new_unit'].area

# Sort the GeoDataFrame by the area, ascending
gdf_sorted = gdf_projected.sort_values(by='area_m2', ascending=True)

# Select the top 200 smallest areas
top_200_smallest = gdf_sorted.head(100)

# Convert the areas to square kilometers and print them
areas_km2 = top_200_smallest['area_m2'] / 1e6

for idx, area in enumerate(areas_km2, 1):
    print(f"Rank {idx}: {area:.70f} km^2")


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_lat_long' is not empty
zero_area_with_latlong = gdf[(gdf.geometry.area == 0) & gdf['geometry_lat_long'].notnull()]

# If there are any units with area 0, plot the combined geometries for the first unit
if not zero_area_with_latlong.empty:
    unit = zero_area_with_latlong.iloc[1]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_lat_long, geom_BG, geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(1,0,0,0.5), (0,0,1,0.9), (0,1,0,0.5)])
    
    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty 'geometry_lat_long'.")


In [ ]:
# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_lat_long' is not empty
zero_area_with_latlong = gdf[(gdf.geometry.area == 0) & gdf['geometry_lat_long'].notnull()]

# Print the 'year' column values for these ro
zero_area_with_latlong['year'].value_counts()

In [ ]:
# Count the number of rows with an area of 0
count_zero_area = (gdf_projected['area_m2'] == 0).sum()

print(f"Number of observations with area of 0: {count_zero_area}")

In [ ]:
print(unit)

In [ ]:
# Function to compute mean and median areas for a geometry column
def compute_area_statistics(df, geometry_column):
    # Convert the specific column to a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry_column)
    
    # Set the current CRS to WGS 84
    gdf.crs = "EPSG:4326"

    # Project to a metric-based CRS (Web Mercator in this case, but this might introduce some area distortion)
    gdf_projected = gdf.to_crs(epsg=3857)

    # Filter out missing/empty geometries
    gdf_projected = gdf_projected[gdf_projected[geometry_column].notna() & ~gdf_projected[geometry_column].is_empty]
    
    # Compute areas in square meters 
    gdf_projected['area'] = gdf_projected[geometry_column].area
    
    # Return mean and median
    return gdf_projected['area'].mean() / 1e6, gdf_projected['area'].median() / 1e6  # Convert to km^2

# Compute for each column
for column in ['geometry_lat_long', 'geometry_BG']:
    mean_area, median_area = compute_area_statistics(df_geographic_unique, column)
    print(f"Mean area for {column}: {mean_area} km^2")
    print(f"Median area for {column}: {median_area} km^2")
    print("-------------------------------")


In [ ]:
# Filter out invalid geometries
gdf_zipcode = gdf_zipcode[gdf_zipcode['geometry_zipcode'].notna()]

# Project each geometry one-by-one to avoid memory error
def safe_transform(geom):
    try:
        return geom.to_crs(epsg=3857)
    except:
        return None

gdf_zipcode['geometry_projected'] = gdf_zipcode['geometry_zipcode'].apply(safe_transform)

# Now compute areas for the projected geometries
gdf_zipcode['area_m2'] = gdf_zipcode['geometry_projected'].area

# Count the number of rows with an area of 0
count_zero_area = (gdf_zipcode['area_m2'] == 0).sum()

print(f"Number of observations in 'geometry_zipcode' with area of 0: {count_zero_area}")



In [ ]:
sum((new_unit_df['new_geographic_unit'].isnull()) | (new_unit_df['new_geographic_unit'] == ''))


In [ ]:
total_counts = new_unit_df['new_geographic_unit'].value_counts().sum()
print(total_counts)

In [ ]:
len(new_unit_df)

In [ ]:
df_geographic_unique['geometry_lat_long'].isnull().sum() + 297957


In [ ]:
# Filter out the ZIPcodes that have shape files for both 2020 and 2010
unique_zipcodes_2020 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2020]['reportedZipCode'])
unique_zipcodes_2010 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2010]['reportedZipCode'])
common_zipcodes = unique_zipcodes_2020.intersection(unique_zipcodes_2010)

# Initialize a DataFrame to store overlap information
overlap_df = pd.DataFrame(columns=['ZIPcode', 'Overlap_Area', 'Original_Area_2010', 'Area_Difference_Change'])

# Convert df_geographic_unique into a GeoDataFrame for geometry operations
gdf = gpd.GeoDataFrame(df_geographic_unique, geometry='geometry_zipcode')

for zipcode in common_zipcodes:
    geometry_2020 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2020)].iloc[0]['geometry_zipcode']
    geometry_2010 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2010)].iloc[0]['geometry_zipcode']

    # Check if either geometry is None before proceeding
    if geometry_2020 is None or geometry_2010 is None:
        continue

    original_area_2010 = geometry_2010.area
    overlap_area = geometry_2010.intersection(geometry_2020).area
    area_diff_change = overlap_area / original_area_2010 if original_area_2010 != 0 else 0  # Handle cases where original area is 0
    
    overlap_df.loc[len(overlap_df)] = {'ZIPcode': zipcode, 
                                       'Overlap_Area': overlap_area, 
                                       'Original_Area_2010': original_area_2010, 
                                       'Area_Difference_Change': area_diff_change}
print(overlap_df["Area_Difference_Change"].mean())



In [ ]:
# Filter out the ZIPcodes that have shape files for both 2000 and 2010
unique_zipcodes_2000 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2000]['reportedZipCode'])
unique_zipcodes_2010 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2010]['reportedZipCode'])
common_zipcodes = unique_zipcodes_2000.intersection(unique_zipcodes_2010)

# Initialize a DataFrame to store overlap information
overlap_df = pd.DataFrame(columns=['ZIPcode', 'Overlap_Area', 'Original_Area_2000', 'Area_Difference_Change'])

# Convert df_geographic_unique into a GeoDataFrame for geometry operations
gdf = gpd.GeoDataFrame(df_geographic_unique, geometry='geometry_zipcode')

for zipcode in common_zipcodes:
    geometry_2000 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2000)].iloc[0]['geometry_zipcode']
    geometry_2010 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2010)].iloc[0]['geometry_zipcode']

    # Check if either geometry is None before proceeding
    if geometry_2000 is None or geometry_2010 is None:
        continue

    original_area_2000 = geometry_2000.area
    overlap_area = geometry_2000.intersection(geometry_2010).area
    area_diff_change = overlap_area / original_area_2000 if original_area_2000 != 0 else 0  # Handle cases where original area is 0
    
    overlap_df.loc[len(overlap_df)] = {'ZIPcode': zipcode, 
                                       'Overlap_Area': overlap_area, 
                                       'Original_Area_2000': original_area_2000, 
                                       'Area_Difference_Change': area_diff_change}
print(overlap_df["Area_Difference_Change"].mean())


In [ ]:
# Filter out the ZIPcodes that have shape files for both 2000 and 2010
unique_zipcodes_2000 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2000]['reportedZipCode'])
unique_zipcodes_2010 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2010]['reportedZipCode'])
common_zipcodes = unique_zipcodes_2000.intersection(unique_zipcodes_2010)

# Initialize a DataFrame to store overlap information
overlap_df = pd.DataFrame(columns=['ZIPcode', 'Overlap_Area', 'Original_Area_2000', 'Area_Difference_Change'])

# Convert df_geographic_unique into a GeoDataFrame for geometry operations
gdf = gpd.GeoDataFrame(df_geographic_unique, geometry='geometry_zipcode')

tolerance = 0.001  # This is an arbitrary value and might need adjustingfor zipcode in common_zipcodes:
for zipcode in common_zipcodes:

    geometry_2000 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2000)].iloc[0]['geometry_zipcode']
    geometry_2010 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2010)].iloc[0]['geometry_zipcode']

    # Check if either geometry is None before proceeding
    if geometry_2000 is None or geometry_2010 is None:
        continue

    # Simplify the geometry before buffering
    simplified_geometry_2000 = geometry_2000.simplify(tolerance)
    
    original_area_2000 = simplified_geometry_2000.area
    buffer_distance = (original_area_2000 * 0.10) ** 0.5  # Compute the buffer distance
    buffered_geometry_2000 = simplified_geometry_2000.buffer(buffer_distance)

    overlap_area = buffered_geometry_2000.intersection(geometry_2010).area
    final_area_2010 = geometry_2010.area
    overlap_to_final_ratio = overlap_area / final_area_2010 if final_area_2010 != 0 else 0  # Handle cases where final area is 0
    
    overlap_df.loc[len(overlap_df)] = {'ZIPcode': zipcode, 
                                       'Overlap_Area': overlap_area, 
                                       'Original_Area_2000': original_area_2000, 
                                       'Area_Difference_Change': overlap_to_final_ratio}  # Adjusted column name to reflect change

print(overlap_df["Area_Difference_Change"].mean())

In [ ]:
# Filter out the ZIPcodes that have shape files for both 2000 and 2010
unique_zipcodes_2000 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2000]['reportedZipCode'])
unique_zipcodes_2010 = set(df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021'] == 2010]['reportedZipCode'])
common_zipcodes = unique_zipcodes_2000.intersection(unique_zipcodes_2010)

# Initialize a DataFrame to store overlap information
overlap_df = pd.DataFrame(columns=['ZIPcode', 'Overlap_Area', 'Original_Area_2000', 'Area_Difference_Change'])

# Convert df_geographic_unique into a GeoDataFrame for geometry operations
gdf = gpd.GeoDataFrame(df_geographic_unique, geometry='geometry_zipcode')

for zipcode in common_zipcodes:

    geometry_2000 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2000)].iloc[0]['geometry_zipcode']
    geometry_2010 = gdf[(gdf['reportedZipCode'] == zipcode) & (gdf['yearOfLoss_1990_2021'] == 2010)].iloc[0]['geometry_zipcode']

    # Check if either geometry is None before proceeding
    if geometry_2000 is None or geometry_2010 is None:
        continue

    original_area_2000 = geometry_2000.area
    overlap_area = geometry_2000.intersection(geometry_2010).area
    final_area_2010 = geometry_2010.area
    overlap_to_final_ratio = overlap_area / final_area_2010 if final_area_2010 != 0 else 0  # Handle cases where final area is 0

    overlap_df.loc[len(overlap_df)] = {'ZIPcode': zipcode, 
                                       'Overlap_Area': overlap_area, 
                                       'Original_Area_2000': original_area_2000, 
                                       'Area_Difference_Change': overlap_to_final_ratio}

print(overlap_df["Area_Difference_Change"].mean())


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_lat_long' is not empty
zero_area_with_latlong = gdf[(gdf.geometry.area == 0) & gdf['geometry_lat_long'].notnull()]

# Calculate the number of zero area values with non-empty zip geometry
num_zero_area_with_latlong = len(zero_area_with_latlong)
print(f"Number of zero area values with non-empty 'geometry_lat_long': {num_zero_area_with_latlong}")

# If there are any units with area 0, plot the combined geometries for the first unit
if not zero_area_with_latlong.empty:
    unit = zero_area_with_latlong.iloc[0]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_lat_long, geom_BG, geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(1,0,0,0.5), (0,0,1,0.9), (0,1,0,0.5)])
    
    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty 'geometry_lat_long'.")


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0 and 'geometry_zipcode' is not empty
zero_area_with_zip = gdf[(gdf.geometry.area == 0) & gdf['geometry_zipcode'].notnull()]

# Calculate the number of zero area values with non-empty zip geometry
num_zero_area_with_zip = len(zero_area_with_zip)
print(f"Number of zero area values with non-empty zip code: {num_zero_area_with_zip}")

# If there are any units with area 0, plot the combined geometries for the first unit
if not zero_area_with_zip.empty:
    unit = zero_area_with_zip.iloc[0]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_lat_long, geom_BG, geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(1,0,0,0.5), (0,0,1,0.9), (0,1,0,0.5)])
    
    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty zip code.")


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new_unit_df, geometry='geometry_new_unit')

# Set the CRS to WGS 84 if it's not already set
gdf.crs = gdf.crs if gdf.crs else "EPSG:4326"

# Filter rows where the area of 'geometry_new_unit' is 0, 'geometry_lat_long' is not empty, and 'geometry_zipcode' is not empty
zero_area_with_latlong_zip = gdf[(gdf.geometry.area == 0) & gdf['geometry_lat_long'].notnull() & gdf['geometry_zipcode'].notnull()]

# Calculate the number of zero area values with non-empty lat-long and zip geometry
num_zero_area_with_latlong_zip = len(zero_area_with_latlong_zip)
print(f"Number of zero area values with non-empty lat-long and zip code: {num_zero_area_with_latlong_zip}")

# If there are any such units, plot the combined geometries for the first unit
if not zero_area_with_latlong_zip.empty:
    unit = zero_area_with_latlong_zip.iloc[0]
    
    # Extracting the geometries for the first unit
    geom_lat_long = unit['geometry_lat_long']
    geom_BG = unit['geometry_BG']
    geom_zipcode = unit['geometry_zipcode']
    
    # Combine the three geometries into one GeoSeries for plotting
    combined_geometries = gpd.GeoSeries([geom_lat_long, geom_BG, geom_zipcode])
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    combined_geometries.plot(ax=ax, color=[(1,0,0,0.5), (0,0,1,0.9), (0,1,0,0.5)])
    
    ax.set_title('Combined Geometries for Single Unit with Zero Area in geometry_new_unit')
    
    plt.show()
else:
    print("No units with area of 0 in 'geometry_new_unit' with non-empty lat-long and zip code.")


In [ ]:
import geopandas as gpd

# Convert the DataFrame to a GeoDataFrame if it's not already one
if not isinstance(df_geographic_unique, gpd.GeoDataFrame):
    df_geographic_unique = gpd.GeoDataFrame(df_geographic_unique)

# Filter rows where 'geometry_lat_long' is empty but 'geometry_zipcode' is not empty
no_latlong_with_zip = df_geographic_unique[df_geographic_unique['geometry_lat_long'].isnull() & df_geographic_unique['geometry_zipcode'].notnull()]

# Calculate the number of such observations
num_no_latlong_with_zip = len(no_latlong_with_zip)
print(f"Number of observations without lat-long but with zip code: {num_no_latlong_with_zip}")


In [ ]:

new_unit_df1 = new_unit_df1.drop_duplicates()

chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(new_unit_df1), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = new_unit_df1.iloc[start:end].copy()
    temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"new_geographic_unit_geometry_{start}_{end}.parquet.gzip", compression='gzip')